In [1]:
import numpy as np
import scipy.linalg as la
import scipy.sparse.linalg as las
import LaviRot as lr
from LaviRot.materials import steel
np.set_printoptions(precision=3)
%matplotlib inline

In [2]:
# n_elements = [i for i in range(2, 10, 2)]
n_elements = [2]
speed = 523.6

In [3]:
shaft_od = 0.08
shaft_id = 0.03
shaft_len = 1.2

In [4]:
steel_ex = lr.Material('steel_ex', rho=7800, E=200e9, Poisson=0.27)

In [5]:
rotors_wn = np.zeros((12, len(n_elements)))
for i, n_el in enumerate(n_elements):
    bearing0 = lr.BearingElement(0, kxx=1e18, cxx=0)
    bearing1 = lr.BearingElement(n_el, kxx=1e18, cxx=0)
    bearings = [bearing0, bearing1]

    shaft = [lr.ShaftElement(shaft_len/n_el, shaft_id, shaft_od, steel_ex, n=i)
             for i in range(n_el)]
    disk0 = lr.DiskElement(n_el//2, steel_ex, width=0.08, i_d=shaft_od, o_d=0.4)
    rotor = lr.Rotor(shaft, [disk0], bearings, w=speed)
    rotors_wn[:, i] = rotor.wd

ValueError: could not broadcast input array from shape (6) into shape (12)

In [8]:
len(rotor._eigen()[0])

24

In [12]:
rotor.wn / (2*np.pi)

array([   54.045,    54.051,   246.168,   360.157,  1026.714,  1029.88 ])

In [7]:
rotor.SPARSE = False

In [25]:
rotor.w = 523.6

In [22]:
rotor.SPARSE

False

In [26]:
rotor.wn

array([  339.578,   339.611,  1546.717,  2262.932,  6451.031,  6470.926])

In [21]:
np.abs(la.eig(rotor.A())[0]/(2*np.pi))

array([  1.235e+08,   1.235e+08,   1.235e+08,   1.235e+08,   1.232e+08,
         1.232e+08,   1.232e+08,   1.232e+08,   1.125e+03,   1.125e+03,
         1.088e+03,   1.088e+03,   1.030e+03,   1.030e+03,   1.027e+03,
         1.027e+03,   3.602e+02,   3.602e+02,   2.462e+02,   2.462e+02,
         5.405e+01,   5.405e+01,   5.405e+01,   5.405e+01])

In [75]:
rotor.wn/(2*np.pi)

array([  54.039,   54.039,  245.014,  352.578,  783.345,  783.345,
        783.345,  783.345,  352.578,  245.014,   54.039,   54.039])

In [77]:
np.abs(las.eigs(rotor.A(), sigma=0, which='LM')[0]/(2*np.pi))

array([  54.039,   54.039,   54.039,   54.039,  245.014,  245.014])

In [76]:
rotor.wd

array([  339.537,   339.537,  1539.468,  2215.314,  4921.898,  4921.898,
       -4921.898, -4921.898, -2215.314, -1539.468,  -339.537,  -339.537])

In [27]:
rotor.SPARSE

False